In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Clear_Cell_Carcinoma"
cohort = "GSE117230"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma/GSE117230"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/GSE117230.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE117230.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE117230.csv"
json_path = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional profiling of paired ccRCC patient CD8+ and CD19+ lymphocytes isolated from tumors, normal tissues, and circulating peripheral blood"
!Series_summary	"Tumor infiltrating lymphocytes are widely associated with positive outcomes, but are still markers of a systemic failed immune response against unresolved cancer. Cancer immunotherapies can reverse their tolerance phenotypes, while preserving tumor-reactivity and neoantigen-specificity shared with circulating immune cells. We performed comprehensive transcriptomic analyses to identify gene signatures common to circulating and tumor infiltrating lymphocytes in the context of clear cell renal cell carcinoma. Modulated genes also associated with disease outcome were validated in other cancer types. Using bioinformatics we identified practical diagnostic markers and actionable targets of the failed immune response. On circulating lymphocytes, three genes could efficiently stratify patient

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# This dataset appears to be about transcriptional profiling, which indicates gene expression data is likely available
is_gene_available = True

# 2.1 Data Availability
# Extract trait information from the sample characteristics dictionary
# The disease state is in row 0, indicating clear cell renal cell carcinoma (ccRCC) status
trait_row = 0

# No age information is provided in the sample characteristics
age_row = None

# No gender information is provided in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert disease state to binary: 1 for ccRCC patient, 0 for healthy control
    """
    if value is None or ":" not in value:
        return None
    
    value = value.split(":", 1)[1].strip().lower()
    
    if "ccrcc patient" in value:
        return 1
    elif "healthy control" in value:
        return 0
    else:
        return None

def convert_age(value):
    """
    Convert age to continuous value
    """
    # Not used in this dataset, but defined for completeness
    return None

def convert_gender(value):
    """
    Convert gender to binary: 1 for male, 0 for female
    """
    # Not used in this dataset, but defined for completeness
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(is_final=False, 
                              cohort=cohort, 
                              info_path=json_path, 
                              is_gene_available=is_gene_available, 
                              is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# We confirmed trait_row is not None, so we proceed with clinical feature extraction
if trait_row is not None:
    # Extract clinical features using the existing clinical_data variable
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM3288712': [0.0], 'GSM3288713': [0.0], 'GSM3288714': [0.0], 'GSM3288715': [0.0], 'GSM3288716': [0.0], 'GSM3288717': [1.0], 'GSM3288718': [1.0], 'GSM3288719': [1.0], 'GSM3288720': [1.0], 'GSM3288721': [1.0], 'GSM3288722': [1.0], 'GSM3288723': [1.0], 'GSM3288724': [1.0], 'GSM3288725': [1.0], 'GSM3288726': [1.0], 'GSM3288727': [1.0], 'GSM3288728': [1.0], 'GSM3288729': [1.0], 'GSM3288730': [1.0], 'GSM3288731': [1.0], 'GSM3288732': [0.0], 'GSM3288733': [0.0], 'GSM3288734': [0.0], 'GSM3288735': [0.0], 'GSM3288736': [0.0], 'GSM3288737': [1.0], 'GSM3288738': [1.0], 'GSM3288739': [1.0], 'GSM3288740': [1.0], 'GSM3288741': [1.0], 'GSM3288742': [1.0], 'GSM3288743': [1.0], 'GSM3288744': [1.0], 'GSM3288746': [1.0], 'GSM3288747': [1.0], 'GSM3288749': [1.0], 'GSM3288751': [1.0], 'GSM3288753': [1.0], 'GSM3288755': [1.0], 'GSM3288757': [1.0]}
Clinical data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE117230.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Transcriptional profiling of paired ccRCC patient CD8+ and CD19+ lymphocytes isolated from tumors, normal tissues, and circulating peripheral blood"
Line 1: !Series_geo_accession	"GSE117230"
Line 2: !Series_status	"Public on Oct 17 2020"
Line 3: !Series_submission_date	"Jul 17 2018"
Line 4: !Series_last_update_date	"Oct 24 2020"
Line 5: !Series_summary	"Tumor infiltrating lymphocytes are widely associated with positive outcomes, but are still markers of a systemic failed immune response against unresolved cancer. Cancer immunotherapies can reverse their tolerance phenotypes, while preserving tumor-reactivity and neoantigen-specificity shared with circulating immune cells. We performed comprehensive transcriptomic analyses to identify gene signatures common to circulating and tumor infiltrating lymphocytes in the context of clear cell renal cell carcinoma. Modulated genes also associated with disease outcome were validated in oth

Successfully extracted gene data with 70523 rows
First 20 gene IDs:
Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data

# The gene identifiers in this dataset appear to be probe IDs (e.g., "2824546_st")
# rather than standard human gene symbols like BRCA1, TP53, etc.

# The suffix "_st" suggests these are likely Affymetrix probe IDs
# These identifiers need to be mapped to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 2891713 rows

Gene annotation preview (first few rows):
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST0000

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify relevant columns for gene mapping
print("Creating gene mapping from annotation data...")

# Looking at the column names and data in gene_annotation, 
# 'ID' appears to match the probe ids in gene_expression data
# 'gene_assignment' contains gene symbol information

# 2. Create a gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')
print(f"Created gene mapping with {len(mapping_df)} rows")

# Preview the mapping
print("\nMapping dataframe preview:")
print(preview_df(mapping_df, n=3))

# 3. Apply gene mapping to convert probe-level data to gene expression data
print("\nApplying gene mapping to convert probe data to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Verify the conversion was successful
print(f"Converted to gene expression data with {len(gene_data)} genes")
print("First few gene symbols:")
print(list(gene_data.index[:10]))

# Save the processed gene data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Creating gene mapping from annotation data...
Created gene mapping with 70753 rows

Mapping dataframe preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1'], 'Gene': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST00000408384 // MIR1302-9 // microRNA 1302-9 // --- // 100422831 /// ENST00000408384 // MIR1302-2 // microRNA 1302-2 // --- // 100302278 /// ENST00000469289 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000469289 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST00000469289 // MIR1302-9 // microRNA 1302-9 // 

Converted to gene expression data with 71528 genes
First few gene symbols:
['A-', 'A-2', 'A-52', 'A-575C2', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0']


Gene expression data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE117230.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Check if gene_data is empty before normalization
    if gene_data.empty:
        print("Gene data is empty. Skipping normalization.")
        normalized_gene_data = gene_data
    else:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data (even if empty, to maintain consistent workflow)
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# 2. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    # Load the clinical data from the previously saved file
    clinical_df = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
    
    # Create proper trait column 
    clinical_df_t = clinical_df.T
    clinical_df_t.reset_index(inplace=True)
    clinical_df_t.columns = [trait] + [col for col in clinical_df_t.columns[1:]]
    clinical_df = clinical_df_t
    
    print(f"Correctly formatted clinical data shape: {clinical_df.shape}")
    print(f"Clinical data columns: {clinical_df.columns.tolist()}")
    is_trait_available = trait in clinical_df.columns
    
    if is_trait_available and not normalized_gene_data.empty:
        # Link the data
        linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Check if we have at least one sample with trait value
        if trait in linked_data.columns:
            trait_count = linked_data[trait].count()
            print(f"Number of samples with trait values: {trait_count}")
            
            if trait_count > 0:
                # 3. Handle missing values systematically
                print("\nHandling missing values...")
                linked_data = handle_missing_values(linked_data, trait)
                print(f"After handling missing values, data shape: {linked_data.shape}")
                
                # Check if we still have samples after missing value handling
                if linked_data.shape[0] > 0:
                    # 4. Determine whether the trait and demographic features are biased
                    print("\nChecking for bias in features...")
                    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                else:
                    print("Error: All samples were removed during missing value handling.")
                    is_biased = True
            else:
                print("No samples have valid trait values. Dataset cannot be used.")
                is_biased = True
        else:
            print(f"Trait column '{trait}' not found in linked data.")
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing.")
        if not is_trait_available:
            print(f"Reason: Trait column '{trait}' is not available in clinical data.")
        if normalized_gene_data.empty:
            print("Reason: Gene expression data is empty.")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 5. Final quality validation
print("\nPerforming final validation...")
note = ""
if 'linked_data' in locals() and linked_data.empty:
    if not normalized_gene_data.empty and not clinical_df.empty:
        note = "Dataset failed in linking phase. Clinical data structure could not be properly matched with gene expression data."
    elif normalized_gene_data.empty:
        note = "Dataset failed in gene mapping phase. Could not properly map probe IDs to gene symbols."
    elif clinical_df.empty:
        note = "Dataset failed in clinical data extraction. No valid trait information available."
elif 'is_biased' in locals() and is_biased:
    note = "Dataset passed initial processing but contains severely biased trait distribution."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=not normalized_gene_data.empty,  # Set based on actual gene data availability
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note=note
)

# 6. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...
After normalization: 24018 unique gene symbols


Normalized gene expression data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE117230.csv

Linking clinical and genetic data...
Loaded clinical data with shape: (1, 40)
Correctly formatted clinical data shape: (40, 2)
Clinical data columns: ['Kidney_Clear_Cell_Carcinoma', 0]
Linked data shape: (42, 24058)
Trait column 'Kidney_Clear_Cell_Carcinoma' not found in linked data.

Performing final validation...
Dataset not usable for Kidney_Clear_Cell_Carcinoma association studies. Data not saved.
